In [1]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os
from glob import glob
import cv2
import numpy as np
from scipy.fft import fft
import pandas as pd

In [1]:
# Load labeled frames and their corresponding labels
def load_labeled_frames(data_folder):
    images = []
    labels = []

    for image_path in glob(os.path.join(data_folder, '*.jpg')):
        image = cv2.imread(image_path)
        image = cv2.resize(image, (30, 30))

        label = os.path.splitext(os.path.basename(image_path))[0].split('_')[-1]

        images.append(image)

        if label == '0':
            labels.append(0)
        elif label == '1':
            labels.append(1)

    return images, labels

# Preprocess the data
def preprocess_data(images, labels):
    images = np.array(images)
    labels = np.array(labels)
    
    # duplicate the images and labels
    images = np.concatenate((images, images), axis=0)
    labels = np.concatenate((labels, labels), axis=0)

    # Normalize pixel values to be between 0 and 1
    images = images / 255.0

    # Shuffle and split the data into training and testing sets
    images, labels = shuffle(images, labels, random_state=46)
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=45)

    return X_train, X_test, y_train, y_test

In [3]:
# Example usage
output_folder = 'labeled_frames_binary_test'

# Load labeled frames and their corresponding labels
images, labels = load_labeled_frames(output_folder)

# Preprocess the data
X_train, X_test, y_train, y_test = preprocess_data(images, labels)

input_shape = X_train[0].shape
print(input_shape)

(30, 30, 3)


In [ ]:
y_train

array([1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0])

In [105]:
# Build the model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',  # Use binary crossentropy for binary classification
              metrics=['accuracy', 'AUC'])

# Train the model
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))


Epoch 1/30
3/3 [==============================] - 3s 323ms/step - loss: 0.6640 - accuracy: 0.5814 - auc: 0.6015 - val_loss: 0.5670 - val_accuracy: 0.6364 - val_auc: 1.0000
Epoch 2/30
3/3 [==============================] - 0s 62ms/step - loss: 0.5365 - accuracy: 0.6512 - auc: 0.8574 - val_loss: 0.4509 - val_accuracy: 0.6364 - val_auc: 1.0000
Epoch 3/30
3/3 [==============================] - 0s 58ms/step - loss: 0.5007 - accuracy: 0.6512 - auc: 0.8664 - val_loss: 0.4183 - val_accuracy: 0.6364 - val_auc: 1.0000
Epoch 4/30
3/3 [==============================] - 0s 57ms/step - loss: 0.4376 - accuracy: 0.6512 - auc: 0.9494 - val_loss: 0.4107 - val_accuracy: 0.6364 - val_auc: 1.0000
Epoch 5/30
3/3 [==============================] - 0s 49ms/step - loss: 0.3880 - accuracy: 0.6512 - auc: 0.9812 - val_loss: 0.3736 - val_accuracy: 0.6364 - val_auc: 1.0000
Epoch 6/30
3/3 [==============================] - 0s 50ms/step - loss: 0.3453 - accuracy: 0.6628 - auc: 0.9881 - val_loss: 0.3375 - val_accuracy

In [58]:
# Evaluate the model
test_loss, test_acc, test_auc = model.evaluate(X_test, y_test, verbose=2)

print('\nTest AUC:', test_auc)

1/1 - 3s - loss: 0.0059 - accuracy: 1.0000 - auc: 1.0000 - 3s/epoch - 3s/step

Test AUC: 1.0


In [2]:
framelist = []
data_folder = 'labeled_frames_fixed'
video = './vidscapstone/vid12_calibration.mp4'

for image_path in glob(os.path.join(data_folder, '*.jpg')):
    framelist.append(image_path)

# framelist = [
#              'labeled_frames_fixed\\vid9.mp4_00054_2.jpg', 
#              'labeled_frames_fixed\\vid9.mp4_00047_0.jpg', 
#              'labeled_frames_fixed\\vid9.mp4_00053_3.jpg', 
#              'labeled_frames_fixed\\vid9.mp4_00046_2.jpg',
#              ]
    
# Load model
model = tf.keras.models.load_model('testing_binary_class_classification/testing_binary_class_classification_model_Adam.h5')

In [3]:
def extract_blinking_lights(img):

    if img is None:
        print(f"Error: Unable to load image from {image_path}")
    else:
        # Convert the image to grayscale
        grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Apply thresholding to highlight the lights
        _, thresholded = cv2.threshold(grey, 200, 255, cv2.THRESH_BINARY)
        
        # Find contours in the thresholded image
        contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Extract ROIs based on contours
        rois = []
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            w = w + 20
            h = h + 20
            x = x - 10
            y = y - 10
            
            rois.append((x,y,w,h))
        
        return rois

def are_similar_coordinates(coord1, coord2):
    x1, y1, w1, h1 = coord1
    x2, y2, w2, h2 = coord2
    
    # Adjust these threshold values based on your requirements
    position_threshold = 50  # Adjust as needed
    size_threshold = 50  # Adjust as needed
    
    # Check if the coordinates are similar in terms of position and size
    position_diff = abs(x1 - x2) + abs(y1 - y2)
    size_diff = abs(w1 - w2) + abs(h1 - h2)
    
    return position_diff < position_threshold and size_diff < size_threshold
    

In [4]:
merged_rois = []
all_rois = []

cap = cv2.VideoCapture(video)

fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Calculate the duration of the video in seconds
video_duration = total_frames / fps

print(f"Video Duration: {video_duration:.2f} seconds")

framelistVideo = []

if not cap.isOpened():
    print("Error: Could not open the video file.")
    exit()

while True:
    ret, frame = cap.read()

    if not ret:
        break

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    rois = extract_blinking_lights(img)
    for roi in rois:
        all_rois.append(roi)
    
    framelistVideo.append(img)

if all_rois is not None:
    for roi_coordinates in all_rois:
        add_to_merged = True

        for merged_roi_coordinates in merged_rois:
            if are_similar_coordinates(roi_coordinates, merged_roi_coordinates):
                add_to_merged = False
                break
        
        if add_to_merged:
            merged_rois.append(roi_coordinates)

Video Duration: 4.83 seconds


In [5]:
# selected_ROIs = []

# while True:
#     # Select and save ROI manually from the frame
#     roi = cv2.selectROI("Select the area", framelistVideo[8], fromCenter=False)

#     # Print the selected bounding box coordinates
#     print('Selected bounding box: {}'.format(roi))
#     selected_ROIs.append(roi)
    
#     # Delay to allow time for 'q' key to register
#     cv2.waitKey(1)
    
#     # Check for 'q' key press to exit the loop
#     if cv2.waitKey(1) == ord('q'):
#         cv2.destroyAllWindows()
#         break

#     cv2.destroyAllWindows()

# if len(selected_ROIs) > 0:
#     merged_rois = selected_ROIs


In [6]:
merged_rois = [(493, 283, 52, 53), (763, 256, 55, 41)]

# Draw the merged ROIs on the frame
for x, y, w, h in merged_rois:
    cv2.rectangle(framelistVideo[8], (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display the frame
cv2.imshow('frame', framelistVideo[8])

# Delay to allow time for 'q' key to register
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
rois_in_frames = []
for img in framelistVideo:
    on_off = []
    breaker = False
    for roi_coordinates in merged_rois:
        x, y, w, h = roi_coordinates
        x = x - 30
        y = y - 30
        w = w + 60
        h = h + 60
        
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Extract the ROI
        roi = img[y:y+h, x:x+w]

        # Convert the ROI to grayscale and apply thresholding
        grey = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        
        # Apply thresholding to highlight the lights
        _, thresholded = cv2.threshold(grey, 180, 255, cv2.THRESH_BINARY)
        
        # Find contours in the thresholded image
        contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if len(contours) == 0:
            on_off.append(0)
        else:
            on_off.append(1)
    rois_in_frames.append(on_off)
print(rois_in_frames)


[[1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1],

In [8]:
# Transpose the array using zip
cal_freq = []
transposed_array = list(zip(*rois_in_frames))
print(transposed_array)
for i, separated_array in enumerate(transposed_array):
    filtered_tuple = [separated_array[0]] 

    for i in range(1, len(separated_array)):
        if separated_array[i] != separated_array[i - 1]:
            filtered_tuple.append(separated_array[i])
            
    frequency = filtered_tuple.count(1)/video_duration
    print(f"Frequency of light {i}: {frequency}")
    cal_freq.append(frequency)

[(1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1), (0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0)]
Frequency of light 144: 5.172413793103448
Frequency of light 144: 4.9655172413793105


In [9]:
print(f'Number of blinking lights: {len(merged_rois)}')

#print all coordinates
for i, coordinates in enumerate(merged_rois):
    print(coordinates)
    #use coordinates to create image roi
    x, y, w, h = coordinates
    #extract from original image
    img = cv2.imread(framelist[0])
    roi = img[y:y+h, x:x+w]
    #display image
    cv2.imshow(f'ROI {i}', roi)

cv2.waitKey(0)
cv2.destroyAllWindows()

Number of blinking lights: 2
(493, 283, 52, 53)
(763, 256, 55, 41)


In [10]:
import cv2
import numpy as np
from scipy.fft import fft
import concurrent.futures

global previous_frame

# Function to process each ROI
def process_roi(roi):

    x, y, w, h = roi
    x = x
    y = y
    w = w
    h = h
    
    roi_processed = frame[y:y+h, x:x+w]
    roi_processed = cv2.resize(roi_processed, (30, 30), interpolation=cv2.INTER_LINEAR)
    roi_processed = cv2.convertScaleAbs(roi_processed)
    roi_processed = np.array(roi_processed)
    roi_processed = roi_processed / 255.0
    roi_processed = np.expand_dims(roi_processed, axis=0)  # Add batch size dimension

    roi_processed_prev = previous_frame[y:y+h, x:x+w]
    roi_processed_prev = cv2.resize(roi_processed_prev, (30, 30), interpolation=cv2.INTER_LINEAR)
    roi_processed_prev = cv2.convertScaleAbs(roi_processed_prev)
    roi_processed_prev = np.array(roi_processed_prev)
    roi_processed_prev = roi_processed_prev / 255.0
    roi_processed_prev = np.expand_dims(roi_processed_prev, axis=0)  # Add batch size dimension
    
    return np.concatenate((roi_processed, roi_processed_prev))


In [12]:
import contextlib
import io
from datetime import datetime

video_path = './vidscapstone/vid12_calibration.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error opening video file")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)  # Get the frames per second from the video

# Initialize roi_frequencies for each ROI
roi_frequencies = {i: [] for i in range(len(merged_rois))}
reset_interval = 90

batch_size = len(merged_rois) * 2
frame_count = 0
frame_total = 0

timer = cv2.getTickCount()

print("Predictions have been started.")
print(merged_rois)
while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame_count += 1
    frame_total += 1

    # Draw the merged ROIs on the frame
    for x, y, w, h in merged_rois:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # show frame
    cv2.imshow('frame', frame)
    # Wait for a key press (0 means wait indefinitely)
    key = cv2.waitKey(0)

    # Break the loop if 'q' is pressed or the video ends
    if key == ord('q'):
        cv2.destroyAllWindows()
        break
    
    cv2.destroyAllWindows()

    if frame_count % 2 != 0 and reset_interval != frame_count:
        previous_frame = frame.copy()
        continue

    # Process ROIs sequentially
    rois_processed = [process_roi(roi) for roi in merged_rois]
    
    rois_processed = np.vstack(rois_processed)  # Stack ROIs to create a batch

    # Make predictions on the batch of ROIs
    with contextlib.redirect_stdout(io.StringIO()):
        predictions = model.predict(rois_processed)
        
    predictions = np.where(predictions > 0.5, 1, 0)
    ordered_predictions_1 = []
    ordered_predictions_2 = []
    
    for i, prediction in enumerate(predictions):
        if i%2 == 0:
            ordered_predictions_1.append(prediction)
        else:
            ordered_predictions_2.append(prediction)

    predictions = np.concatenate((ordered_predictions_1, ordered_predictions_2))
    print(predictions)
    # Distribute predictions to each ROI
    for i, prediction in enumerate(predictions):
        roi_index = i % len(merged_rois)
        if len(roi_frequencies[roi_index]) == 0 or roi_frequencies[roi_index][-1] != prediction[0]:
            roi_frequencies[roi_index].append(prediction[0])

    # Reset frequencies after a certain number of frames
    if frame_count % reset_interval == 0:
        seconds_passed = frame_count / fps
        print(f"Seconds passed: {seconds_passed:.2f}")
        # Calculate the average frequency for each ROI
        for i in range(len(merged_rois)):
            print(roi_frequencies[i])
            print(seconds_passed)
            if len(roi_frequencies[i]) > 0:
                frequency = sum(roi_frequencies[i]) / seconds_passed
                print(f"Frequency of light {i}: {frequency:.2f} Hz")
                if frequency < cal_freq[i] - 0.7 or frequency > cal_freq[1] + 0.7:
                    print(f"Inconsistencies on light {i} detected! Please check the light.")
                    print(f"The calculated Frequency is {frequency:.2f} Hz, while the expected frequency is {cal_freq[i]:.2f} Hz.")
                    with open('logs.txt', 'a') as f:
                        f.write(f"Date & Time: {datetime.now().strftime('%d/%m/%Y, %H:%M:%S')} \nWarning: Expected frequency of light {i}: {cal_freq[i]:.2f}\nActual frequency: {frequency:.2f} Hz.\n\n")
                        f.close()

        roi_frequencies = {i: [] for i in range(len(merged_rois))}
        frame_count = 0
        print("Predicting next batch of frames...")

# Calculate FPS
fps = frame_total/((cv2.getTickCount() - timer) / cv2.getTickFrequency())
print(f"FPS: {fps}")

# Calculate total time
total_time = (cv2.getTickCount() - timer) / cv2.getTickFrequency()
print(f"Total time: {total_time} seconds")
cap.release()

Predictions have been started.
[(493, 283, 52, 53), (763, 256, 55, 41)]


[[1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]]
FPS: 0.43923061804855584
Total time: 15.9383953 seconds


In [13]:
import contextlib
import io
from datetime import datetime

video_path = './vidscapstone/vid12_calibration.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error opening video file")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)  # Get the frames per second from the video

# Initialize roi_frequencies for each ROI
roi_frequencies = {i: [] for i in range(len(merged_rois))}
reset_interval = 90

batch_size = len(merged_rois) * 2
frame_count = 0
frame_total = 0

timer = cv2.getTickCount()

print("Predictions have been started.")
print(merged_rois)
while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame_count += 1
    frame_total += 1
    
    # Process ROIs sequentially
    rois_to_batch = []
    
    for r in merged_rois:
        x, y, w, h = r

        roi_processed = frame[y:y+h, x:x+w]
        cv2.imshow('frame', roi_processed)
        # Wait for a key press (0 means wait indefinitely)
        key = cv2.waitKey(0)

        # Break the loop if 'q' is pressed or the video ends
        if key == ord('q'):
            cv2.destroyAllWindows()
            break
        
        cv2.destroyAllWindows()
        roi_processed = cv2.resize(roi_processed, (30, 30), interpolation=cv2.INTER_LINEAR)
        roi_processed = cv2.convertScaleAbs(roi_processed)
        roi_processed = np.array(roi_processed)
        roi_processed = roi_processed / 255.0
        roi_processed = np.expand_dims(roi_processed, axis=0)  # Add batch size dimension

        rois_to_batch.append(roi_processed)
            
    rois_to_batch = np.vstack(rois_to_batch)
    predictions = model.predict(rois_to_batch)
    predictions = np.where(predictions > 0.5, 1, 0)
    print(predictions)
    # Distribute predictions to each ROI
    for i, prediction in enumerate(predictions):
        roi_index = i % len(merged_rois)
        if len(roi_frequencies[roi_index]) == 0 or roi_frequencies[roi_index][-1] != prediction[0]:
            roi_frequencies[roi_index].append(prediction[0])
    processed_frames = []
    # Reset frequencies after a certain number of frames
    if frame_count % reset_interval == 0:
        seconds_passed = frame_count / fps
        print(f"Seconds passed: {seconds_passed:.2f}")
        # Calculate the average frequency for each ROI
        for i in range(len(merged_rois)):
            print(seconds_passed)
            if len(roi_frequencies[i]) > 0:
                frequency = sum(roi_frequencies[i]) / seconds_passed
                print(f"Frequency of light {i}: {frequency:.2f} Hz")
                if frequency < cal_freq[i] - 0.7 or frequency > cal_freq[1] + 0.7:
                    print(f"Inconsistencies on light {i} detected! Please check the light.")
                    print(f"The calculated Frequency is {frequency:.2f} Hz, while the expected frequency is {cal_freq[i]:.2f} Hz.")
                    with open('logs.txt', 'a') as f:
                        f.write(f"Date & Time: {datetime.now().strftime('%d/%m/%Y, %H:%M:%S')} \nWarning: Expected frequency of light {i}: {cal_freq[i]:.2f}\nActual frequency: {frequency:.2f} Hz.\n\n")
                        f.close()

        roi_frequencies = {i: [] for i in range(len(merged_rois))}
        frame_count = 0
        print("Predicting next batch of frames...")

# Calculate FPS
fps = frame_total/((cv2.getTickCount() - timer) / cv2.getTickFrequency())
print(f"FPS: {fps}")

# Calculate total time
total_time = (cv2.getTickCount() - timer) / cv2.getTickFrequency()
print(f"Total time: {total_time} seconds")
cap.release()

Predictions have been started.
[(493, 283, 52, 53), (763, 256, 55, 41)]
1/1 [==============================] - 0s 121ms/step
[[1]
 [1]]
1/1 [==============================] - 0s 99ms/step
[[1]
 [1]]
1/1 [==============================] - 0s 108ms/step
[[1]
 [1]]
1/1 [==============================] - 0s 124ms/step
[[1]
 [1]]
1/1 [==============================] - 0s 108ms/step
[[1]
 [1]]
1/1 [==============================] - 0s 87ms/step
[[1]
 [1]]
1/1 [==============================] - 0s 118ms/step
[[1]
 [1]]
1/1 [==============================] - 0s 53ms/step
[[1]
 [1]]
1/1 [==============================] - 0s 82ms/step
[[1]
 [1]]
1/1 [==============================] - 0s 72ms/step
[[1]
 [1]]
1/1 [==============================] - 0s 121ms/step
[[1]
 [1]]
1/1 [==============================] - 0s 90ms/step
[[1]
 [1]]


ValueError: need at least one array to concatenate